In [1]:
import numpy as np
import pandas as pd
np.random.seed(42)

In [2]:
train = pd.read_csv('./mnist_train_small.csv')
test = pd.read_csv('./mnist_test.csv')
train.head()

,6,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,0.581,0.582,0.583,0.584,0.585,0.586,0.587,0.588,0.589,0.590
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
train.shape

(19999, 785)

In [4]:
test.shape

(9999, 785)

In [26]:
X_train = np.array(train.iloc[:,1:])
y_train = train['6'].to_numpy().reshape(-1,1)

In [27]:
A0 = np.copy(X_train.T)
A0.shape

(784, 19999)

In [28]:
y_train.shape

(19999, 1)

In [37]:
def initialize_parameters(n_x, n_h, n_y): # (number of inputs, number of units in hidden, number of outputs)
  # Need 2 sets of parameters for 2-layer network
  W1 = np.random.randn(n_h, n_x) * 0.01
  b1 = np.zeros((n_h, 1))
  W2 = np.random.randn(n_y, n_h) * 0.01
  b2 = np.zeros((n_y, 1))

  parameters = {
      'W1': W1,
      'b1': b1,
      'W2': W2,
      'b2': b2
  }
  return parameters

In [38]:
def relu(Z):
  return np.maximum(0,Z)

In [50]:
def sigmoid(Z):
  sigmoid = 1 / (1 + np.exp(-Z))
  return sigmoid

In [51]:
def forward_propagation(X):
  W1, b1, W2, b2 = initialize_parameters(A0.shape[0], 18, 9).values()
  Z1 = np.dot(W1, X) + b1
  A1 = relu(Z1)
  Z2 = np.dot(W2, A1) + b2
  A2 = sigmoid(Z2)

  cache = {
      'Z1': Z1,
      'A1': A1,
      'Z2': Z2,
      'A2': A2
  }
  return A2, cache

In [55]:
A2, cache = forward_propagation(A0)
A2.shape

(9, 19999)

In [56]:
def compute_cost(A2, Y):
  m = Y.shape[0]
  cross_entropy = np.dot(np.log(A2),Y) + np.dot(np.log(1-A2), 1-Y)
  cost = -np.sum(cross_entropy) / m
  return cost

In [57]:
compute_cost(A2, y_train)

16.129066994254043

In [59]:
# dA2 = dL / dA2 = -y/a - (1-y)/(1-a)
# dz2 = (dL/ dA2)(dA2 / dZ2) = 